In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import csv



def update_df(csv_file_path, url):
    # Laden und Verarbeiten der bestehenden CSV-Datei
    existing_data = []
    try:
        with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                existing_data.append(row)
    except FileNotFoundError:
        print("Error: File Not Found where Data is saved on PC")

    # Webseitenabruf und Analyse mit BeautifulSoup
    req = Request(url = url, headers={"user-agent": "my-app"})
    response = urlopen(req)

    html = BeautifulSoup(response, 'html')
    insider_table = html.find(class_="body-table insider-trading-table")

    # Vergleich mit den bestehenden Daten und Hinzufügen neuer Einträge zur CSV-Datei
    new_data_flag = False
    for row in insider_table.find_all("tr")[1:]:
        insider_name = row.find('td')
        # Get the Tradername in the first td element
        if insider_name is not None:
            insider_name = insider_name.text.strip()
        else:
            insider_name = None
        # Get the title of the Insider in the second td element
        insider_title = row.find_all('td')[1]
        if insider_title is not None:
            insider_title = insider_title.text.strip()
        else:
            insider_title = None
        # Get the date of the transaction in the third td element
        transaction_date = row.find_all('td')[2]
        if transaction_date is not None:
            transaction_date = transaction_date.text.strip()
        else:
            transaction_date = None
        # Get the type of transaction in the fourth td element
        transaction_type = row.find_all('td')[3]
        if transaction_type is not None:
            transaction_type = transaction_type.text.strip()
        else:
            transaction_type = None
        # Get the Cost of transaction in the fifth td element
        cost = row.find_all('td')[4]
        if cost is not None:
            cost = cost.text.strip()
        else:
            cost = None
        # Get the num of shares in the sixth td element
        num_shares = row.find_all('td')[5]
        if num_shares is not None:
            num_shares = num_shares.text.strip()
        else:
            num_shares = None
        # Get the Value of shares in the seventh td element
        value_shares = row.find_all('td')[6]
        if value_shares is not None:
            value_shares = value_shares.text.strip()
        else:
            value_shares = None
        # Get the num of shares the insider owns in the eighth td element
        total_num_shares_insider_owns = row.find_all('td')[7]
        if total_num_shares_insider_owns is not None:
            total_num_shares_insider_owns = total_num_shares_insider_owns.text.strip()
        else:
            total_num_shares_insider_owns = None
        # Get the date of the SEC Form, ka was das ist,  the ninth td element
        SEC_date = row.find_all('td')[7]
        if SEC_date is not None:
            SEC_date = SEC_date.text.strip()
        else:
            SEC_date = None
        # Überprüfe, ob der Eintrag bereits in den bestehenden Daten vorhanden ist
        if [insider_name, insider_title, transaction_date, transaction_type, cost, 
            num_shares, value_shares, total_num_shares_insider_owns, SEC_date] not in existing_data:
        # Füge den Eintrag hinzu
            existing_data.append([insider_name, insider_title, transaction_date, transaction_type, cost, 
            num_shares, value_shares, total_num_shares_insider_owns, SEC_date])
            new_data_flag = True
    
    # Speichern der aktualisierten Daten in der CSV-Datei wenn neue Daten vorhanden
    if new_data_flag == True:
        try:
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(existing_data)
        except FileNotFoundError as e:
            print("Error: Could not write data into" + str(csv_file_path))

#update_df(path, root)